In [1]:
from nlpia.data.loaders import get_data

/home/nixie/anaconda3/envs/nlpiaenv/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [29]:
movies = get_data('hutto_movies')

In [30]:
movies.head().round(3)

,sentiment,text
id,,
1,2.267,The Rock is destined to be the 21st Century's ...
2,3.533,The gorgeously elaborate continuation of ''The...
3,-0.600,Effective but too tepid biopic
4,1.467,If you sometimes like to go to the movies to h...
5,1.733,"Emerges as something rare, an issue movie that..."


In [31]:
movies.describe().round(2)

,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [32]:
import pandas as pd
pd.set_option('display.width', 75)

In [33]:
from nltk.tokenize import casual_tokenize
from collections import Counter
bow = []
for text in movies.text:
    bow.append(Counter(casual_tokenize(text)))

In [34]:
df_bow = pd.DataFrame.from_records(bow)
df_bow = df_bow.fillna(0).astype(int)
df_bow.shape

(10605, 20756)

In [35]:
df_bow.head()[list(bow[0].keys())]

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Schwarzenegger,",",Jean,Claud,Van,Damme,or,Steven,Segal,.
0,1,1,1,1,2,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [36]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb = nb.fit(df_bow, movies.sentiment > 0)


In [42]:
nb

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [46]:
movies['predicted_sentiment'] = nb.predict(df_bow) * 8 - 4

In [62]:
movies.head()

,sentiment,text,predicted_sentiment,error,sentiment_is_positive,predicted_is_positive
id,,,,,,
1,2.266667,The Rock is destined to be the 21st Century's ...,4,1.733333,1,1
2,3.533333,The gorgeously elaborate continuation of ''The...,4,0.466667,1,1
3,-0.600000,Effective but too tepid biopic,-4,3.400000,0,0
4,1.466667,If you sometimes like to go to the movies to h...,4,2.533333,1,1
5,1.733333,"Emerges as something rare, an issue movie that...",4,2.266667,1,1


### Average mean error (MAE)

In [48]:
movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs()

In [49]:
movies.error.mean().round(1)

2.4

In [55]:
movies['sentiment_is_positive'] = (movies.sentiment > 0).astype(int)
movies['predicted_is_positive'] = (movies.predicted_sentiment > 0).astype(int)

In [58]:
movies['''sentiment predicted_sentiment sentiment_is_positive predicted_is_positive'''.split()].head()

,sentiment,predicted_sentiment,sentiment_is_positive,predicted_is_positive
id,,,,
1,2.266667,4,1,1
2,3.533333,4,1,1
3,-0.600000,-4,0,0
4,1.466667,4,1,1
5,1.733333,4,1,1


### Accuracy

In [60]:
(movies.predicted_is_positive == movies.sentiment_is_positive).sum()/len(movies)

0.9344648750589345

### Testing on a new data set

In [65]:
products = get_data('hutto_products')
bow = []
for text in products.text:
    bow.append(Counter(casual_tokenize(text)))

In [70]:
df_products_bow = pd.DataFrame.from_records(bow)
df_products_bow = df_products_bow.fillna(0).astype(int)
df_all_bow = df_bow.append(df_products_bow)


In [81]:
df_products_bow.head()

,!,"""",#,$,%,&,',(,),*,...,zips,zombie,zombies,zone,zoning,zzzzzzzzz,½,élan,–,’
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
df_products_bow = df_all_bow.iloc[len(movies):][df_bow.columns]
df_products_bow = df_products_bow.fillna(0).astype(int)

In [73]:
df_products_bow.shape

(3546, 20756)

In [74]:
df_bow.shape

(10605, 20756)

In [82]:
products['ispos'] = (products.sentiment > 0).astype(int)

In [83]:
products['predicted_is_pos'] = nb.predict(df_products_bow.values).astype(int)

In [84]:
products.head()

,id,sentiment,text,ispos,predicted_is_pos
0,1_1,-0.90,troubleshooting ad-2500 and ad-2600 no picture...,0,0
1,1_2,-0.15,"repost from january 13, 2004 with a better fit...",0,0
2,1_3,-0.20,does your apex dvd player only play dvd audio ...,0,0
3,1_4,-0.10,or does it play audio and video but scrolling ...,0,0
4,1_5,-0.50,before you try to return the player or waste h...,0,0


In [86]:
(products.predicted_is_pos == products.ispos).sum()/len(products)

0.5572476029328821